In [ ]:
import warnings
import igraph as ig
import networkx as nx
import collections
import scipy as sp
import numpy as np
import bMatching as bm
import importlib
import matplotlib.pylab as plt
import time

In [ ]:
def read_graphml(f1,f2,fl=None,input_dir=None):
    
    if input_dir is not None:
        f1=input_dir+f1
        f2=input_dir+f2
        
    
    G1 = ig.read(f1,format="graphml")
    G2 = ig.read(f2,format="graphml")
    
    L=None
    if fl is not None:
        if input_dir is not None:
            fl=input_dir+fl
        L=ig.read(fl,format="graphml")
    
    return G1,G2,L

In [ ]:
def read_networks(f1,f2,fl=None,input_dir=None):
    
    if input_dir is not None:
        f1=input_dir+f1
        f2=input_dir+f2
        
    T=nx.read_leda(f1)
    nx.write_graphml(T,'graph.graphml')
    G1 = ig.read('graph.graphml',format="graphml")
    
    T=nx.read_leda(f2)
    nx.write_graphml(T,'graph.graphml')
    G2 = ig.read('graph.graphml',format="graphml")
    
    L=None
    if fl is not None:
        if input_dir is not None:
            fl=input_dir+fl
        L=ig.read(fl,format="graphml")
    
    return G1,G2,L

In [ ]:
def get_alignment(G1,G2,M):
    
    n1=G1.vcount()
    n2=G2.vcount()
    Align=[]
    for i in range(len(M)):
        if i < n1:
            j=M[i][0][1]-n1
            
            u=G1.vs[i]['id']
            v=G2.vs[j]['id']
            
            Align.append((u,v))
        else:
            break
    
    return Align 

In [ ]:
def save_alignment(L,filename):
    
    fl=open(filename,"w")
    for (u,v) in L:
        t=u+" "+v+"\n"
        fl.write(t)
    
    fl.close()

In [ ]:
def round_heuristic(L,perm,w):
    
    wk=np.concatenate((np.ravel(w),np.ravel(w)))
    
    for i in range(int(L.nnz/2),L.nnz):
        wk[i]=wk[perm[i]]
    
    L.data=wk
    
    return bm.bSuitor(L,1)
    

In [ ]:
def evaluate(M1,M2,bestM):
    
    AL=get_alignment(G1,G2,M)
    save_alignment(AL,res_dir+resf)
    
    return bestM

In [ ]:
def othermax(L,perm,y,z):
    
    ##### First do the row side
    
    w=np.concatenate((np.ravel(z),np.ravel(z)))
    
    for i in range(int(L.nnz/2),L.nnz):
        w[i]=w[perm[i]]
    
    L.data=w
    
    row,col=L.nonzero()
    
    omax_y=[]
    for i in range(int(L.nnz/2)):
        t=L.getrow(row[i]).data
        temp=list(np.sort(t[(-t).argsort()[:2]]))
        
        if len(temp) == 2:
            second =temp[0]
            maxr=temp[1]
        else:
            second=0
            maxr=temp[0]
        
        if second < 0:
            second=0
        if maxr < 0:
            maxr=0
        
        if w[i]==maxr:
            omax_y.append(second)
        else:
            omax_y.append(maxr)
    
    s=y.shape
    omax_y=np.array(omax_y)
    omax_y=np.matrix(omax_y.reshape(s))
    
    #print(z,type(z),z.shape)
    #print(omax_y,type(omax_y))
    #print(L)
    
    
    ##### The column side
    
    w=np.concatenate((np.ravel(y),np.ravel(y)))
    
    for i in range(int(L.nnz/2),L.nnz):
        w[i]=w[perm[i]]
    
    L.data=w
    
    row,col=L.nonzero()
    
    omax_z=[]
    for i in range(int(L.nnz/2),L.nnz):
        t=L.getrow(row[i]).data         #### Since L is symmetric we can use row here insted of col
        temp=list(np.sort(t[(-t).argsort()[:2]]))
        
        if len(temp) == 2:
            second =temp[0]
            maxc=temp[1]
        else:
            second=0
            maxc=temp[0]
        
        if second < 0:
            second=0
        if maxc < 0:
            maxc=0
        
        if w[i]==maxc:
            omax_z.append(second)
        else:
            omax_z.append(maxc)
    
    
    s=z.shape
    omax_z=np.array(omax_z)
    omax_z=np.matrix(omax_z.reshape(s))
    
    #print(y)
    #print(omax_z)
    #print(L)
    
    
    return omax_y,omax_z

In [ ]:
def create_overlaps_matrix(G1,G2,L=None):
    
    ### G1: network 1
    ### G2: network 2
    ### L : Bipartite graph where 
    ### left side is G1 vertices and 
    ### right side is G2 vertices
    
    n1=G1.vcount()
    n2=G2.vcount()
    
    ### for iterating in L 
    G1_vid=list(range(n1))
    G2_vid=list(range(n1,n1+n2))
    
    ###
    nS=0
    if L is None:  
        ### L is not avaiable so assume complete bipartite
        ### and assume edges are indexed accordingly
        nS=n1*n2 
        
    else:
        nS=L.ecount()
    
    #S=ig.Graph()  
    #S.add_vertices(list(range(nS)))
    edgeS=[]
    
    for u in G1_vid:
        
        nbor1=G1.neighbors(u)
        
        for v in G2_vid:
            uv=-1
            if L is not None:
                uv=L.get_eid(u,v,directed=False,error=False)
                
                #if  uv == -1:
                    #print("Sparse L!!")
                    #continue
            else:
                uv= u*n2+(v-n1) #### Be careful
            
            if uv== -1:
                continue ## Just sanity check
            
            nbor2=G2.neighbors(v-n1)
            

            for i in nbor1:
                for jj in nbor2:
                    j=jj+n1 ### shifting the vertex id for bipartite graph

                    ij=-1
                    if L is not None:
                        ### Now check whether the neighbors has cross edge
                        ij=L.get_eid(i,j,directed=False,error=False)

                        #if ij == -1:
                            #print("L Sparse 2 !!")
                            #continue
                    else:
                        ij=(i*n2)+jj
                    
                    if ij==-1:
                        continue ### Just sanity check

                    
                    edgeS.append((uv,ij))

    #S.add_edges(edgeS)
    
    #edges = self.get_edgelist() 
    weights = [1] * len(edgeS)
    S = sp.sparse.csr_matrix((weights, zip(*edgeS)), shape=(nS, nS))
    S = S + sp.sparse.triu(S, 1).T + sp.sparse.tril(S, -1).T
    S=S/2
    
    if L is None:
        row=[]
        col=[]
        data=[1]*(n1*n2)
        for i in range(n1):
            for j in range(n2):
                row.append(i)
                col.append(j+n1)
        row  = np.array(row)
        col  = np.array(col)
        data = np.array(data)
        L = sp.sparse.coo_matrix((data, (row, col)), shape=(n1+n2, n1+n2)) 
    else:
        edgeL=L.get_edgelist()
        nL=L.vcount()
        weights=[1]*len(edgeL)
        L = sp.sparse.csr_matrix((weights, zip(*edgeL)), shape=(nL, nL))
        L = L + sp.sparse.triu(L, 1).T + sp.sparse.tril(L, -1).T
        L=sp.sparse.coo_matrix(L)

    return S,L


In [ ]:
def netAlign(f1,f2,fl=None,resf=None,alpha=1,beta=2,gamma=0.99,maxiter=100,nbatch=20,input_dir=None,res_dir=None):
    
    if input_dir is None:
        input_dir="/home/khan242/netAlign/data/synthetic_networks/"
    if res_dir is None:
        res_dir="/home/khan242/netAlign/results/"  
     
    if resf is None:
        t=f1.split(".")
        resf=t[0]
    
        t=f2.split(".")
        resf=resf+"_"+t[0]+".aln"
    
        resp=resf.split(".")
    
    #### Output
    bestM=[]
    Mbase={}
    
    #### Reading networks
    t1=time.perf_counter()
    #G1,G2,L=read_graphml(f1,f2,fl,input_dir)
    G1,G2,L=read_networks(f1,f2,fl,input_dir)
    t2=time.perf_counter()
    
    print('Network Reading Done: ',round((t2-t1),2))
    S,L=create_overlaps_matrix(G1,G2,L)
    t3=time.perf_counter()
    print('Computing Overlap Matrix: ',round((t3-t2),2))
    
    ts=time.perf_counter()
    ###### Processing L 
    ll=int(L.nnz/2)
    row,cols=L.nonzero()
    perm=list(range(L.nnz))
    
    for i in range(ll,L.nnz):
        c=row[i]
        r=cols[i]
        for j in range(ll):
            if r==row[j] and c==cols[j]:
                perm[i]=j
                break
            else:
                perm[i]=-1 
    
    
    w=np.matrix(L.data[0:ll]).T
      
    
    ###### First iteration
    
    l3=time.perf_counter()
    F=sp.sparse.csr_matrix(S*beta)  ### Line 3
    
    d=alpha*w+F.sum(1)       ### Line 4
    l4=time.perf_counter()
    
    yk=np.matrix(d)          ### Line 5
    zk=np.matrix(d)          ### Line 6
    l5=time.perf_counter()
    
    Sk=sp.sparse.diags(np.ravel(yk+zk-d))*S - F ### Line 7
    
    
    yk1=gamma*yk  ### Line 8
    zk1=gamma*zk
    Sk1=gamma*Sk
    l6=time.perf_counter()
    
    Mbase[0]=round_heuristic(L,perm,yk) ### Line 10
    Mbase[1]=round_heuristic(L,perm,zk) ### Line 11
    
    batch_id=2
    te=time.perf_counter()
    print('Iteration 1 : ',round((te-ts),2),round((l4-l3),2),round((l5-l4),2),round((l6-l5),2),round((te-l6),2))
    for t in range(500):
        
        if t >= maxiter:
            break
        ts=time.perf_counter()
        
        l3=time.perf_counter()
        tdata=list((beta*S+Sk.T).data)
        tdata1 = [max(ele, 0) for ele in tdata] #### Lower bounding to 0
        tdata2 = [min(ele, beta) for ele in tdata1] #### upper bounding beta
        
        #F=sp.sparse.csr_matrix(beta*S+Sk.T)   ### This statement is not needed since structure remains same
        F.data=np.array(tdata2)  ### Line 3
        
        d=alpha*w+F.sum(1)                   ### Line 4
        l4=time.perf_counter()
        
        omax_y,omax_z=othermax(L,perm,yk1,zk1)
        yk=np.matrix(d) - omax_y                    ### Line 5
        zk=np.matrix(d) - omax_z                    ### Line 6
        l5=time.perf_counter()
        
        Sk=sp.sparse.diags(np.ravel(yk+zk-d))*S - F ### Line 7
        

        yk1=gamma*yk + (1-gamma)*yk1 ### Line 8
        zk1=gamma*zk + (1-gamma)*zk1
        Sk1=gamma*Sk + (1-gamma)*Sk1
        l6=time.perf_counter()

        Mbase[batch_id]=round_heuristic(L,perm,yk) ### Line 10
        Mbase[batch_id+1]=round_heuristic(L,perm,zk) ### Line 11
        
        te=time.perf_counter()
        print('Iteration',(t+2),': ',round((te-ts),2),round((l4-l3),2),round((l5-l4),2),round((l6-l5),2),round((te-l6),2))
        
        batch_id=batch_id+2
        if batch_id == nbatch:
            batch_id=0
            #bestM=evaulate(Mbase,bestM)
        
    AL=get_alignment(G1,G2,Mbase[4])
    save_alignment(AL,res_dir+resf)
    return AL

In [ ]:
if __name__ == '__main__':
    
    warnings.filterwarnings("ignore")
    
    
    input_dir="/home/khan242/netAlign/data/synthetic_networks/"
    res_dir="/home/khan242/netAlign/results/"
    
    f1="yeast0_Y2H1.gw"
    f2="yeast10_Y2H1.gw"
    resf="yeast0_yeast10_Y2H1.aln"
    
    netAlign(f1,f2,resf,None,"ALL",input_dir,res_dir)
    
    f1="yeast0_Y2H1.gw"
    f2="yeast15_Y2H1.gw"
    resf="yeast0_yeast15_Y2H1.aln"
    
    netAlign(f1,f2,resf,None,"ALL",input_dir,res_dir)
    
    f1="yeast0_Y2H1.gw"
    f2="yeast20_Y2H1.gw"
    resf="yeast0_yeast20_Y2H1.aln"
    
    netAlign(f1,f2,resf,None,"ALL",input_dir,res_dir)
    
    f1="yeast0_Y2H1.gw"
    f2="yeast25_Y2H1.gw"
    resf="yeast0_yeast25_Y2H1.aln"
    
    netAlign(f1,f2,resf,None,"ALL",input_dir,res_dir)

In [ ]:
#input_dir="/home/khan242/netAlign/data/synthetic networks/"
#input_dir="/home/khan242/netAlign/data/real world networks/"
#res_dir="/home/khan242/netAlign/src/"
importlib.reload(bm)



f1="yeast0_Y2H1.gw"
f2="yeast5_Y2H1.gw"

resf="yeast0_yeast5_Y2H1.aln"    
netAlign(f1,f2,None,resf,maxiter=6,res_dir=res_dir)

In [ ]:
AL=netAlign('g1.graphml','g2.graphml','L.graphml',maxiter=4)
print(AL)

In [ ]:
T=nx.read_leda('../data/synthetic networks/yeast0_Y2H1.gw')

In [ ]:
input_dir="/home/khan242/netAlign/data/synthetic_networks/"



f1="yeast0_Y2H1.gw"
f2="yeast5_Y2H1.gw"
fl="L_0.05_yeast0_yeast5.graphml"
G1,G2,L=read_networks(f1,f2,fl,input_dir)

In [ ]:
G1.vs[0]

In [ ]:
G2.vs[0]

In [ ]:
L.vs[0]

In [ ]:
L.get_eid(0,1769)

In [ ]:
L.es[49]

In [ ]:
print(L.es[0],L.es[0].source,L.es[0].target)

In [ ]:
import pandas as pd